# Agent for Sale

#### Importing important libraries

In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import logging
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import NotFittedError
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools import Function
from pydantic import ConfigDict

In [2]:
load_dotenv()
logging.basicConfig(level=logging.INFO)

In [3]:
data = pd.DataFrame({
    'StoreID': [1, 2, 3, 1],
    'ProdName': ['Product A', 'Product B', 'Product C', 'Product D'],
    'TotalStockQuantity': [100, 50, 80, 60],
    'TotalSalesQuantity': [120, 30, 90, 70]
})

In [4]:
data

,StoreID,ProdName,TotalStockQuantity,TotalSalesQuantity
0,1,Product A,100,120
1,2,Product B,50,30
2,3,Product C,80,90
3,1,Product D,60,70


In [4]:
class StockAnalysisTool(Function):
    name: str = "StockAnalysisTool"
    description: str = "Analyzes stock levels and provides recommendations for inventory balancing."
    data: pd.DataFrame

    model_config = ConfigDict(arbitrary_types_allowed=True)

    def __call__(self, input):
        try:
            logging.info("Starting stock analysis...")

            if self.data is None or self.data.empty:
                logging.error("Data is empty or None.")
                return "No data provided for analysis."

            required_columns = {'TotalStockQuantity', 'TotalSalesQuantity', 'StoreID', 'ProdName'}
            missing_columns = required_columns - set(self.data.columns)
            if missing_columns:
                logging.error(f"Missing columns: {missing_columns}")
                return f"Data is missing required columns: {', '.join(missing_columns)}"

            df = self.data.copy()
            df['StockShortage'] = df['TotalStockQuantity'] - df['TotalSalesQuantity']
            df['ShortageFlag'] = (df['StockShortage'] < 0).astype(int)

            unique_classes = df['ShortageFlag'].unique()
            if len(unique_classes) == 1:
                logging.warning("Only one class present in data. Using rule-based recommendations.")
                recommendations = []
                for idx, row in df.iterrows():
                    status = 'Shortage' if row['ShortageFlag'] == 1 else 'Sufficient Stock'
                    recommendation = f"StoreID: {int(row['StoreID'])}, Product: {row['ProdName']}, Status: {status}"
                    recommendations.append(recommendation)
                result = "\n".join(recommendations)
                return result if result else "No recommendations generated."

            features = df[['TotalStockQuantity', 'TotalSalesQuantity']]
            labels = df['ShortageFlag']

            X_train, X_test, y_train, y_test = train_test_split(
                features, labels, test_size=0.2, random_state=42, stratify=labels
            )

            if X_test.empty or X_train.empty:
                logging.error("Empty train or test set after splitting.")
                return "Not enough data to perform train-test split."

            model = LogisticRegression()
            model.fit(X_train, y_train)
            predictions = model.predict(X_test)

            X_test = X_test.reset_index(drop=True)
            test_indices = X_test.index

            recommendations = []
            for i in range(len(predictions)):
                status = 'Shortage' if predictions[i] == 1 else 'Sufficient Stock'
                idx = test_indices[i]
                store_id = int(df.iloc[idx]['StoreID'])
                prod_name = df.iloc[idx]['ProdName']
                recommendation = f"StoreID: {store_id}, Product: {prod_name}, Status: {status}"
                recommendations.append(recommendation)

            result = "\n".join(recommendations)
            logging.info("Stock analysis completed successfully.")

            return result if result else "No recommendations generated."

        except Exception as e:
            logging.exception("An error occurred during stock analysis.")
            return f"An error occurred during stock analysis: {str(e)}"

In [5]:
stock_analysis_tool = StockAnalysisTool(data=data)

In [6]:
api_key = os.getenv('GROQ_API_KEY')
if not api_key:
    raise ValueError("API key not found. Please set the GROQ_API_KEY environment variable.")

In [7]:
stock_agent = Agent(
    name='Stock Agent',
    model=Groq(id="llama-3.1-8b-Instant", api_key=api_key),
    tools=[stock_analysis_tool],
    instructions="You are an AI assistant helping to balance inventory across multiple stores. Analyze the provided stock data and give clear, actionable recommendations for inventory balancing.",
    show_tool_calls=True,
    markdown=True
)

In [8]:
# Test the __call__ method independently
test_output = stock_analysis_tool(input="Test input")
print("Test Output from StockAnalysisTool:")
print(test_output)

INFO:root:Starting stock analysis...
ERROR:root:An error occurred during stock analysis.
Traceback (most recent call last):
  File "C:\Users\eDominer\AppData\Local\Temp\ipykernel_16732\1914624785.py", line 44, in __call__
    X_train, X_test, y_train, y_test = train_test_split(
                                       ^^^^^^^^^^^^^^^^^
  File "c:\Users\eDominer\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\eDominer\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py", line 2801, in train_test_split
    train, test = next(cv.split(X=arrays[0], y=stratify))
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\eDominer\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py", line 1843, in split
    for train, test in self._iter_indices(X, y, groups):
  File "c:\Users\eDominer\anaconda3\Lib\site-packages\sklearn\model_selection\_spl

Test Output from StockAnalysisTool:
An error occurred during stock analysis: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.


In [9]:
# Run the agent
user_query = "Please analyze the stock data and provide recommendations."
response = stock_agent.run(input=user_query)
print("\nAgent's Response:")
print(response)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 400 Bad Request"


BadRequestError: Error code: 400 - {'error': {'message': "'messages.2' : for 'role:tool' the following must be satisfied[('messages.2.content' : Value is not nullable)]", 'type': 'invalid_request_error'}}

# New Agent

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = {
    'StoreID': [1, 1, 2, 2, 3, 3],
    'ProdName': ['Product A', 'Product B', 'Product A', 'Product B', 'Product A', 'Product B'],
    'TotalStockQuantity': [100, 200, 350, 80, 90, 60],
    'TotalSalesQuantity': [110, 140, 190, 70, 100, 50]
}

df = pd.DataFrame(data)

In [3]:
class StockRecommendationAgent:
    def __init__(self, data):
        self.data = data
        self.recommendations = []

    def analyze_stock(self):
        self.data['StockShortage'] = self.data['TotalStockQuantity'] - self.data['TotalSalesQuantity']
        return self.data
    
    def identify_low_high_stock(self):
        low_stock_threshold = 0
        high_stock_threshold = 120

        low_stock = self.data[self.data['StockShortage'] < low_stock_threshold]
        high_stock = self.data[self.data['StockShortage'] > high_stock_threshold]

        return low_stock, high_stock
    
    def generate_recommendations(self, low_stock, high_stock):
        for _, low in low_stock.iterrows():
            product = low['ProdName']
            low_store_id = low['StoreID']
            
            matching_high_stock = high_stock[high_stock['ProdName'] == product]
            
            if not matching_high_stock.empty:
                for _, high in matching_high_stock.iterrows():
                    high_store_id = high['StoreID']
                    self.recommendations.append({
                        'LowStockStoreID': low_store_id,
                        'HighStockStoreID': high_store_id,
                        'Product': product,
                        'Recommendation': f'Transfer stock from Store {high_store_id} to Store {low_store_id} for Product {product}'
                    })
                    
    def get_recommendations(self):
        return self.recommendations

In [4]:
agent = StockRecommendationAgent(df)

In [5]:
agent.data

,StoreID,ProdName,TotalStockQuantity,TotalSalesQuantity
0,1,Product A,100,110
1,1,Product B,200,140
2,2,Product A,350,190
3,2,Product B,80,70
4,3,Product A,90,100
5,3,Product B,60,50


In [7]:
agent.analyze_stock()

,StoreID,ProdName,TotalStockQuantity,TotalSalesQuantity,StockShortage
0,1,Product A,100,110,-10
1,1,Product B,200,140,60
2,2,Product A,350,190,160
3,2,Product B,80,70,10
4,3,Product A,90,100,-10
5,3,Product B,60,50,10


In [8]:
low_stock, high_stock = agent.identify_low_high_stock()

In [9]:
low_stock

,StoreID,ProdName,TotalStockQuantity,TotalSalesQuantity,StockShortage
0,1,Product A,100,110,-10
4,3,Product A,90,100,-10


In [10]:
high_stock

,StoreID,ProdName,TotalStockQuantity,TotalSalesQuantity,StockShortage
2,2,Product A,350,190,160


In [11]:
agent.generate_recommendations(low_stock, high_stock)

In [13]:
recommendations = agent.get_recommendations()
for rec in recommendations:
    print(f'Recommendation: {rec["Recommendation"]}')

Recommendation: Transfer stock from Store 2 to Store 1 for Product Product A
Recommendation: Transfer stock from Store 2 to Store 3 for Product Product A
